In [1]:
import random
from math import inf


params = {
    "farmer_productivity": 1.5,
    "land_capacity": 120,

    "steal_rate": 3,
    "suppression_prob": 0.3,
    "confiscate_rate": 0.7,

    "tax_rate": 1.5,
    "soldier_base_pay": 2.0,

    "alpha_perception": 0.25,
    "fear_increment": 0.1,
    "fear_decay": 0.2,

    "utility_margin": 0.5,
    "switch_urge_threshold": 2.0,
}

EPS = 1e-6
FEAR_MAX = 10.0


# ------------------ PERSON CLASS ------------------ #

class Person:
    def __init__(self, role, wealth=0.0, prod_mult=1.0, aggr_mult=1.0, skill_mult=1.0):
        self.role = role
        self.wealth = max(0.0, wealth)

        # Psychology
        self.fear = 0.0
        self.perceived_violence = 0.0
        self.loyalty = random.uniform(0.8, 1.2)

        # Switching state
        self.next_role = role
        self.age_in_role = 0
        self.switch_cooldown = 3

        # Last-tick flows (mainly for logging, not utilities now)
        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

        # Heterogeneity
        self.prod_mult = random.uniform(0.9, 1.1)
        self.aggr_mult = random.uniform(0.9, 1.1)
        self.skill_mult = random.uniform(0.9, 1.1)

    # --- housekeeping ---
    def reset_for_new_tick(self):
        self.next_role = self.role
        self.age_in_role += 1
        if self.switch_cooldown > 0:
            self.switch_cooldown -= 1
        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

    # --- role helpers ---
    def is_farmer(self):  return self.role == "farmer"
    def is_bandit(self):  return self.role == "bandit"
    def is_soldier(self): return self.role == "soldier"

    # --- actions ---
    def produce(self, farmer_productivity, scale=1.0):
        if not self.is_farmer():
            return 0.0
        inc = max(0.0, self.prod_mult * farmer_productivity * max(0.0, scale))
        self.wealth = max(0.0, self.wealth + inc)
        self.last_production = inc
        return inc

    def attempt_rob(self, victim, steal_rate, fear_bump_robbed=0.3):
        if not (self.is_bandit() and victim and victim.role == "farmer"):
            return 0.0, False

        stolen = min(
            max(0.0, self.aggr_mult * steal_rate),
            max(0.0, victim.wealth),
        )
        if stolen <= 0:
            return 0.0, False

        victim.wealth = max(0.0, victim.wealth - stolen)
        victim.fear = min(FEAR_MAX, victim.fear + max(0.0, fear_bump_robbed))

        self.wealth = max(0.0, self.wealth + stolen)
        self.last_rob_gain = stolen
        return stolen, True

    def attempt_suppress(self, target_bandit, p_hit, confiscate_rate, fear_bump_suppressed=0.2):
        if not (self.is_soldier() and target_bandit and target_bandit.role == "bandit"):
            return 0.0, False

        hit_prob = max(0.0, min(1.0, self.skill_mult * p_hit))
        if random.random() >= hit_prob:
            return 0.0, False

        amt = min(
            max(0.0, self.skill_mult * confiscate_rate),
            max(0.0, target_bandit.wealth),
        )

        target_bandit.wealth = max(0.0, target_bandit.wealth - amt)
        target_bandit.fear = min(FEAR_MAX, target_bandit.fear + fear_bump_suppressed)

        self.wealth = max(0.0, self.wealth + amt)
        return amt, True

    def pay_tax(self, tax_rate):
        if not self.is_farmer():
            return 0.0
        tax_rate = max(0.0, min(1.0, tax_rate))
        tax = tax_rate * max(0.0, self.wealth)
        self.wealth = max(0.0, self.wealth - tax)
        return tax

    def receive_pay(self, amount):
        if amount > 0:
            self.wealth = max(0.0, self.wealth + amount)
            self.last_pay = amount

    # --- psychology ---
    def update_psychology(self, global_violence, alpha, fear_inc, fear_dec):
        a = max(0.0, min(1.0, alpha))
        gv = max(0.0, global_violence)

        # smoothed perception of violence
        self.perceived_violence = (1 - a) * self.perceived_violence + a * gv

        if self.perceived_violence > 0.5:
            self.fear = min(FEAR_MAX, self.fear + fear_inc)
        else:
            self.fear = max(0.0, self.fear - fear_dec)

    # --- forward-looking utility prediction ---
    def predicted_utility(self, role, world, params):
        """
        Predict this agent's utility NEXT TICK if they were 'role',
        given current world composition and last-tick violence.
        """

        # approx populations if THIS person changes role
        f = world.farmer_pop
        b = world.bandit_pop
        s = world.soldier_pop

        # remove self from current role
        if self.is_farmer():
            f -= 1
        elif self.is_bandit():
            b -= 1
        elif self.is_soldier():
            s -= 1

        # add self as 'role'
        if role == "farmer":
            f += 1
        elif role == "bandit":
            b += 1
        elif role == "soldier":
            s += 1

        violence = world.total_violence_last_tick

        # ---- farmer ----
        if role == "farmer":
            if f > 0:
                scale = min(1.0, params["land_capacity"] / (f + EPS))
            else:
                scale = 1.0
            predicted_prod = self.prod_mult * params["farmer_productivity"] * scale
            safety_bonus = max(0.0, 0.5 - 0.02 * violence)
            return predicted_prod + safety_bonus - 0.01 * b

        # ---- bandit ----
        if role == "bandit":
            if f > 0:
                base_rob = self.aggr_mult * params["steal_rate"]
            else:
                base_rob = 0.0
            return (
                base_rob
                - 0.02 * b                     # competition
                + 0.03 * max(0, f - 10)        # more farmers => more loot
                - 0.02 * s                     # soldiers make life worse
            )

        # ---- soldier ----
        if role == "soldier":
            # rough expected tax pool next tick:
            # farmers * expected prod * tax_rate
            if f > 0:
                scale = min(1.0, params["land_capacity"] / (f + EPS))
                avg_prod = params["farmer_productivity"] * scale
                tax_pool = f * avg_prod * params["tax_rate"]
            else:
                tax_pool = 0.0

            base_total = params["soldier_base_pay"] * s
            if s > 0:
                pay_per = (tax_pool + base_total) / s
            else:
                pay_per = params["soldier_base_pay"]

            return pay_per + 0.02 * b - 0.30   # soldier disutility

        return -999.0

    # --- switching with 5-tick lockout ---
    def decide_switch(self, utilities, switch_urge_threshold, utility_margin, global_tick):
        # first few ticks: no switching at all
        if global_tick <= 5:
            self.next_role = self.role
            return

        if self.switch_cooldown > 0:
            self.next_role = self.role
            return

        urge = self.fear / (self.loyalty + EPS)
        current_u = utilities.get(self.role, -1e9)

        # if urge is low, they don't even consider switching
        if urge <= switch_urge_threshold:
            self.next_role = self.role
            return

        # choose best role by predicted utility
        best_role = self.role
        best_u = current_u
        for r, u in utilities.items():
            if u > best_u:
                best_role, best_u = r, u

        if best_u > current_u + utility_margin:
            self.next_role = best_role
        else:
            self.next_role = self.role

    def finalize_role_switch(self):
        if self.next_role != self.role:
            self.role = self.next_role
            self.age_in_role = 0
            self.switch_cooldown = 1  # small hysteresis

    # --- export for logging ---
    def to_dict(self):
        return {
            "role": self.role,
            "wealth": round(self.wealth, 3),
            "fear": round(self.fear, 3),
            "perceived_violence": round(self.perceived_violence, 3),
            "loyalty": round(self.loyalty, 3),
            "age_in_role": self.age_in_role,
        }

